In [ ]:
%%capture
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

### Question 3: Connecting to the Kafka server

In [ ]:
import json
import time 
from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

### Question 4: Sending data to the stream

In [ ]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

#### Sending the taxi data

In [ ]:
import pandas as pd

usecols = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount']
df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', compression='gzip', usecols=usecols)

### Question 5: Sending the Trip Data

In [ ]:
t0 = time.time()

topic_name = 'green-trips'


for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)
    
producer.flush()
    
t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

### Question 6: Parsing the data

In [ ]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()


green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()
query.stop()

In [ ]:
query.lastProgress

In [ ]:
from pyspark.sql import types
from pyspark.sql import functions as F


schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())


green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [ ]:
green_stream \
    .writeStream \
    .format("console") \
    .start()

### Question 7: Most popular destination

In [ ]:
popular_destinations = (green_stream
    .withColumn('timestamp', F.current_timestamp())
    .groupBy(F.window(F.col("timestamp"), "5 minutes"), "DOLocationID")
    .count()
    .orderBy(F.desc('count'))
)

query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination(20) # 20 seconds timeout

In [ ]:
lookup_df = pd.read_csv('taxi_zone_lookup.csv')
lookup_df[lookup_df['LocationID'] == 74]

In [ ]:
!sudo rm green_tripdata_2019-10.csv.gz
!sudo rm taxi_zone_lookup.csv